# ResNet contineud
- Load prev trained model, more epochs, decrease learning rate

In [1]:
!pip install iterative-stratification

In [2]:
import tensorflow as tf
print("GPUs Available:", len(tf.config.list_physical_devices('GPU')))


2025-10-08 13:11:24.780568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759929085.046579      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759929085.116304      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPUs Available: 0


2025-10-08 13:11:43.675338: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import cv2
import os
import sys
from io import StringIO
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications.resnet50 as resnet
import warnings
warnings.filterwarnings('ignore')

In [4]:
print(os.listdir("/kaggle/input"))

# Path to competition dataset
data_dir = "/kaggle/input/grand-xray-slam-division-b"
# Check what files are inside
print('Filenames of the data', os.listdir(data_dir))

['resnetfinetune', 'resnetfinetune3epoch', 'grand-xray-slam-division-b', 'newresnet1']
Filenames of the data ['test2', 'sample_submission_2.csv', 'train2.csv', 'train2']


In [5]:
# Load the training CSV metadata with labels
train = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/train2.csv")

print('Metadata shape:',train.shape)
train.head()

Metadata shape: (108494, 21)


,Image_name,Patient_ID,Study,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000003_001_001.jpg,3,1,Male,41.0,Frontal,AP,0,1,0,...,1,0,0,1,0,0,0,0,0,0
1,00000004_001_001.jpg,4,1,Female,20.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00000004_001_002.jpg,4,1,Female,20.0,Lateral,Lateral,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,00000006_001_001.jpg,6,1,Female,42.0,Frontal,AP,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,00000010_001_001.jpg,10,1,Female,50.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
# 1. Feature & Target Preperation
# Define labels
conditions = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]
# Features you want
features = ["ViewCategory", "ViewPosition", "Age", "Sex"]

# Encode categorical features
from sklearn.preprocessing import LabelEncoder

train_enc = train.copy()   # train data encoded
for col in ["ViewCategory", "ViewPosition", "Sex"]:  # features that can be encoded
    le = LabelEncoder()
    train_enc[col] = le.fit_transform(train[col].astype(str))

X = train_enc[features].values
y = train[conditions].values

In [7]:
print(X.shape) # 4 features (ViewCategory, ViewPosition, Age, Sex)
print(y.shape)  # 14 conditions

(108494, 4)
(108494, 14)


In [8]:
# 2. Adding ViewBalancing for Stratification: ViewCategory= Frontal, Lateral; since ViewCategory is unbalanced

# One-hot encode ViewCategory and append to 
view_onehot = pd.get_dummies(train["ViewCategory"], prefix="view").values

y_aug = np.hstack([y, view_onehot])  # augmented target matrix (added ViewCategory as y to stratify and reduce bias)

In [9]:
y_aug.shape

(108494, 16)

# MSKF

In [10]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# 3. Multilabel Stratified K-Fold Split
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y_aug)):
    print(f"Fold {fold}")
    print(" Train:", len(train_idx), " Val:", len(val_idx))

    train_df = train.iloc[train_idx].reset_index(drop=True)
    val_df   = train.iloc[val_idx].reset_index(drop=True)

    # Check condition + view balance
    print("  Train views:", train_df["ViewCategory"].value_counts().to_dict())
    print("  Val views:", val_df["ViewCategory"].value_counts().to_dict())
    print("  Train labels sum:", train_df[conditions].sum().to_dict())
    print("  Val labels sum:", val_df[conditions].sum().to_dict())
    print("-"*60)
    
    folds.append((train_df, val_df))

# Use first fold for training
train_df, val_df = folds[0]
print(f"✅ Selected Fold 1 — Train {train_df.shape}, Val {val_df.shape}")

Fold 0
 Train: 86795  Val: 21699
  Train views: {'Frontal': 76011, 'Lateral': 10784}
  Val views: {'Frontal': 19003, 'Lateral': 2696}
  Train labels sum: {'Atelectasis': 30890, 'Cardiomegaly': 27984, 'Consolidation': 23715, 'Edema': 21254, 'Enlarged Cardiomediastinum': 30053, 'Fracture': 11662, 'Lung Lesion': 9885, 'Lung Opacity': 39217, 'No Finding': 27391, 'Pleural Effusion': 27655, 'Pleural Other': 5544, 'Pneumonia': 11453, 'Pneumothorax': 6991, 'Support Devices': 29908}
  Val labels sum: {'Atelectasis': 7723, 'Cardiomegaly': 6996, 'Consolidation': 5929, 'Edema': 5313, 'Enlarged Cardiomediastinum': 7513, 'Fracture': 2916, 'Lung Lesion': 2472, 'Lung Opacity': 9805, 'No Finding': 6974, 'Pleural Effusion': 6914, 'Pleural Other': 1387, 'Pneumonia': 2863, 'Pneumothorax': 1747, 'Support Devices': 7477}
------------------------------------------------------------
Fold 1
 Train: 86795  Val: 21699
  Train views: {'Frontal': 76011, 'Lateral': 10784}
  Val views: {'Frontal': 19003, 'Lateral': 

# data generator

In [11]:
class XRayDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, img_size=(224, 224), is_test=False, **kwargs):
        super().__init__(**kwargs)
        self.dataframe = dataframe.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.is_test = is_test
        self.image_dir = '/kaggle/input/grand-xray-slam-division-b/train2/' if not is_test else '/kaggle/input/grand-xray-slam-division-b/test2/'
        self.conditions = conditions
        
        if not os.path.exists(self.image_dir):
            print(f"Error: Directory {self.image_dir} not found.")
            raise FileNotFoundError(f"Directory {self.image_dir} missing.")
    
    def __len__(self):
        return (len(self.dataframe) + self.batch_size - 1) // self.batch_size
    
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min(start + self.batch_size, len(self.dataframe))
        batch_data = self.dataframe.iloc[start:end]
        
        images, labels = [], []
        
        for _, row in batch_data.iterrows():
            img_path = os.path.join(self.image_dir, row['Image_name'])
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            
            if img is not None and img.shape[0] > 0 and img.shape[1] > 0:
                img = cv2.resize(img, self.img_size)
                img = resnet.preprocess_input(img)
                images.append(img)
                
                if not self.is_test:
                    labels.append(row[self.conditions].values.astype(np.float32))
        
        if not images:
            dummy_img = np.zeros((*self.img_size, 3), dtype=np.float32)
            images.append(dummy_img)
            if not self.is_test:
                labels.append(np.zeros(len(self.conditions), dtype=np.float32))
        
        if not self.is_test:
            return np.array(images), np.array(labels)
        else:
            return np.array(images)

# Create generators
batch_size = 32
train_generator = XRayDataGenerator(train_df, batch_size=batch_size)
val_generator = XRayDataGenerator(val_df, batch_size=batch_size)
print("Data generators created.")

# Also define test_df (important for submission)
test_df = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/sample_submission_2.csv")
test_df["Image_name"] = test_df["Image_name"].astype(str)

print("✅ Data generators ready — Train, Val, and Test loaded.")

Data generators created.
✅ Data generators ready — Train, Val, and Test loaded.


# resnet improved 

In [12]:
# from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50

def build_resnet_model(num_classes=14):
    # Load Resnet with cached ImageNet weights
    base_model = ResNet50(weights="imagenet",  include_top=False, input_shape=(224, 224, 3))
    
    print("✅ Weights loaded successfully.")
    base_model.trainable = False   # freeze backbone for now
    
    # add custom head
    inputs = base_model.input
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)  # multilabel
    
    model = Model(inputs, outputs)
    print("model created successfully")
    return model
    
# load resnet baseline model with AUC=0.87 public final_resnet_model.h5
from tensorflow.keras.models import load_model
model = build_resnet_model(num_classes=14)
model.load_weights("/kaggle/input/newresnet1/keras/default/1/final_resnet_model.h5")

model.compile(
    optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(name="auc")]
)

print("Model Architecture: ResNet50 + Custom Head")
print(f"Total parameters: {model.count_params():,}")
trainable_params = sum([tf.size(v).numpy() for v in model.trainable_variables])
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {model.count_params() - trainable_params:,}")
print("Model compiled successfully!")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
✅ Weights loaded successfully.
model created successfully
Model Architecture: ResNet50 + Custom Head
Total parameters: 24,115,854
Trainable parameters: 528,142
Non-trainable parameters: 23,587,712
Model compiled successfully!


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

callbacks = [
    ModelCheckpoint("resnet_finetune_epoch_{epoch:02d}_valAUC_{val_auc:.4f}.weights.h5", save_weights_only=True, save_best_only=False,
                    monitor="val_auc", mode="max", verbose=1),
    ReduceLROnPlateau(monitor="val_auc", factor=0.5, patience=2, mode="max", verbose=1),
    EarlyStopping(monitor="val_auc", patience=5, mode="max", restore_best_weights=True)
]
model = build_resnet_model(num_classes=14)
model.load_weights("/kaggle/input/resnetfinetune3epoch/resnet_finetune.weights.h5")   # continue training from last best weights
# 🔧 Compile before training
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
print("✅ Model compiled successfully!")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    initial_epoch=3,  # resume from where we left off when loading last best weights
    epochs=5,
    verbose=1,
    callbacks=callbacks
)
val_auc = history.history.get("val_auc", [0])[-1]
print(f"✅ Continued fine-tuning done — new Validation AUC ≈ {val_auc:.4f}")
model.save("final_resnet_finetuned.h5")

from IPython.display import FileLink
FileLink("final_resnet_finetuned.h5")


✅ Weights loaded successfully.
model created successfully
✅ Model compiled successfully!
Epoch 4/5
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9030 - loss: 0.3274
Epoch 4: saving model to resnet_finetune_epoch_04_valAUC_0.9099.weights.h5
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 10118s 4s/step - auc: 0.9030 - loss: 0.3274 - val_auc: 0.9099 - val_loss: 0.3209 - learning_rate: 1.0000e-04
Epoch 5/5
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9043 - loss: 0.3253
Epoch 5: saving model to resnet_finetune_epoch_05_valAUC_0.9100.weights.h5
2713/2713 ━━━━━━━━━━━━━━━━━━━━ 9790s 4s/step - auc: 0.9043 - loss: 0.3253 - val_auc: 0.9100 - val_loss: 0.3192 - learning_rate: 1.0000e-04
✅ Continued fine-tuning done — new Validation AUC ≈ 0.9100


/kaggle/working/final_resnet_finetuned.h5

In [14]:
import h5py
f = h5py.File("/kaggle/input/resnetfinetune3epoch/resnet_finetune.weights.h5", "r")
print(list(f.keys()))

['layers', 'optimizer', 'vars']


In [15]:
# ----------------------------------------
# 🔮 Generate predictions and submission (baseline 3 epochs + 3 new epochs of ResNet50 model using MSKF partial fold)
# ----------------------------------------
print("\nGenerating predictions for submission...")
# load best checkpoint for inference
# model = build_resnet_model(num_classes=14)
# model.load_weights("/kaggle/input/resnetfinetune3epoch/resnet_finetune.weights.h5")  # 3+1 epochs model
# print("✅ Full model loaded successfully!")

# define test_df (important for submission)
test_df = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/sample_submission_2.csv")
test_df["Image_name"] = test_df["Image_name"].astype(str)

test_generator = XRayDataGenerator(test_df, batch_size=32, is_test=True)
preds = model.predict(test_generator, verbose=1)

submission = pd.DataFrame(preds, columns=conditions)
submission.insert(0, "Image_name", test_df["Image_name"].values) # start, column, 
submission.to_csv("ResNet1submission.csv", index=False)

print("✅ Submission file created successfully: submission.csv")
print(submission.head())


Generating predictions for submission...
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 4844s 3s/step
✅ Submission file created successfully: submission.csv
             Image_name  Atelectasis  Cardiomegaly  Consolidation     Edema  \
0  00000002_002_001.jpg     0.559122      0.546943       0.416455  0.382842   
1  00000002_001_001.jpg     0.743050      0.576383       0.456502  0.241495   
2  00000002_001_002.jpg     0.714518      0.533089       0.613938  0.385422   
3  00000008_001_001.jpg     0.885486      0.754806       0.631565  0.463744   
4  00000008_002_001.jpg     0.638492      0.635211       0.570083  0.528030   

   Enlarged Cardiomediastinum  Fracture  Lung Lesion  Lung Opacity  \
0                    0.687344  0.452735     0.141841      0.686748   
1                    0.716566  0.548882     0.044555      0.754154   
2                    0.719916  0.647671     0.155505      0.834968   
3                    0.887164  0.304774     0.036391      0.928583   
4                    0.758540  0.